In [1]:
try:
    from vllm import LLM, SamplingParams
    LOCAL = True
    MODEL_PATH = "deepseek-ai/deepseek-math-7b-rl"
    from functions import *
    dtype = 'auto'
    gpu_memory_utilization = 0.9
except:
    %pip uninstall -y torch -q
    %pip install --no-index --find-links=/kaggle/input/vllm-whl -U vllm -q
    from vllm import LLM, SamplingParams
    LOCAL = False
    MODEL_PATH = "/kaggle/input/deepseek-math"
    dtype = 'half'
    gpu_memory_utilization = 0.99
    from functions_math import *
    import gc
    import multiprocessing

import torch
import pandas as pd

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2024.3.1 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
dask-cuda 23.8.0 requires pynvml<11.5,>=11.0.0, but you have pynvml 11.5.0 which is incompatible.
raft-dask 23.8.0 requires dask==2023.7.1, but you have dask 2024.3.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


2024-04-18 17:29:59,636	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
model_dict = {
          'model':MODEL_PATH,
          'dtype':dtype,
          'enforce_eager':True,
          'gpu_memory_utilization':gpu_memory_utilization,
#           'swap_space':6,
          'max_model_len':2048,
          'kv_cache_dtype':"fp8_e5m2",
          'tensor_parallel_size':1
}

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
repeats = 1
# normalize = False
stop_words = [tokenizer.eos_token]
sampling_params = SamplingParams(n = repeats, best_of=repeats,
                                 temperature=0,
                                 max_tokens=1500,
                                 stop=stop_words)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
NUM_GPUS = 2
def run_inference_one_gpu(gpu_id, prompt_list, model_dict, sampling_params):
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    llm = LLM(**model_dict)
    return llm.generate(prompt_list, sampling_params)

# Splits a list into roughly equally sized pieces
# split_list(["a", "b", "c", "d", "e", "f", "g"], 3) -> [['a', 'b'], ['c', 'd'], ['e', 'f', 'g']]
split_list = lambda l, n: [l[i * len(l) // n: (i + 1) * len(l) // n] for i in range(n)]

def run_inference_multi_gpu(prompts, model_dict, sampling_params):
    split_prompts = split_list(prompts, NUM_GPUS)
    inputs = [(i, p, model_dict, sampling_params) for i, p in enumerate(split_prompts)]

    with multiprocessing.Pool(processes=NUM_GPUS) as pool:
        results = pool.starmap(run_inference_one_gpu, inputs)

    outputs = []
    for result in results:
        outputs.extend(result)

    return outputs

In [4]:
if LOCAL:
    import json
    with open('../Data/AMC/aime_normal.json', 'r') as file:
        data = json.load(file)
    # to have consistent format as in Kaggle
    data = pd.DataFrame(data)
    data.rename(columns={'question': 'problem'}, inplace=True)
else:
    data = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/test.csv')
    if len(data) < 5:
        data = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
        PRIVATE = False
    else:
        PRIVATE = True

In [5]:
# prepare inputs
inputs = []
for index, row in data.iterrows():
    problem = row['problem']
    query_prompt = gen_prompt(problem)
    messages = [{"role": "user","content": query_prompt}]
    input = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs.append(input)

# generation
raw_outputs = run_inference_multi_gpu(inputs, model_dict, sampling_params)

# parse outputs
outs = []
for i, (index, row) in enumerate(data.iterrows()):
    problem = row['problem']
    decoded_output = raw_outputs[i].outputs[0].text
    try:
        answer = decoded_output.split('\n')[-1]
        answer = naive_parse(answer) % 1000
    except:
        answer = 'parsing error'

    if LOCAL:
        outs.append((problem,decoded_output,answer,int(row['final_answer'][0])))
    else:
        outs.append(37 if answer == 'parsing error' else answer)
        torch.cuda.empty_cache()
        gc.collect()
        if not PRIVATE:
            print(decoded_output)
            print(f'\nanswer is {answer}')    

WARNING 04-18 17:30:01 config.py:767] Casting torch.bfloat16 to torch.float16.
WARNING 04-18 17:30:01 config.py:767] Casting torch.bfloat16 to torch.float16.
INFO 04-18 17:30:01 config.py:381] Using fp8_e5m2 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. But it may cause slight accuracy drop. Currently we only support fp8 without scaling factors and make e5m2 as a default format.
INFO 04-18 17:30:01 config.py:381] Using fp8_e5m2 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. But it may cause slight accuracy drop. Currently we only support fp8 without scaling factors and make e5m2 as a default format.
INFO 04-18 17:30:01 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='/kaggle/input/deepseek-math', tokenizer='/kaggle/input/deepseek-math', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, down

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-18 17:30:03 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 04-18 17:30:03 selector.py:25] Using XFormers backend.
INFO 04-18 17:30:03 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 04-18 17:30:03 selector.py:25] Using XFormers backend.
INFO 04-18 17:32:04 model_runner.py:104] Loading model weights took 12.8725 GB
INFO 04-18 17:32:05 model_runner.py:104] Loading model weights took 12.8725 GB
INFO 04-18 17:32:06 gpu_executor.py:94] # GPU blocks: 177, # CPU blocks: 1092
INFO 04-18 17:32:06 gpu_executor.py:94] # GPU blocks: 177, # CPU blocks: 1092


Processed prompts: 100%|██████████| 5/5 [01:34<00:00, 18.90s/it]


The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. This means that the solutions of the equation $kx^2 - 2kx + l - 4 = 0$ are the x-coordinates of points $A$ and $B$. Let's call these solutions $x_1$ and $x_2$. We know that the distance between $A$ and $B$ is 6, so $|x_2 - x_1| = 6$. By Vieta's formulas, we know that $x_1 + x_2 = \frac{2k}{k} = 2$ and $x_1x_2 = \frac{l - 4}{k}$.

Now, we want to find the sum of the squares of the distances from $A$ and $B$ to the origin. The distance from a point $(x, y)$ to the origin is $\sqrt{x^2 + y^2}$. So the square of the distance from point $A$ (with x-coordinate $x_1$) to the origin is $x_1^2 + (kx_1^2 - 2kx_1 + l)^2 = x_1^2 + k^2x_1^4 - 4k^2x_1^3 + 4k^2x_1^2 + (l - 2kx_1)^2$. Similarly, the square of the distance from point $B$ to the origin is $x_2^2 + k^2x_2^4 - 4k^2x_2^3 + 4k^2x_2^2 + (l - 2kx_2)^2$.

The sum of the squares of the distances from $A$ and $B$ to the origin is then
\[(x_1^2 + k^2x_1^4 - 4

In [6]:
if LOCAL:
    outs_df = pd.DataFrame(outs,columns=['problem','output','yhat','y'])
    print(f"correct: {sum(outs_df.yhat == outs_df.y)}")
    print(f"parse error: {sum(outs_df.yhat =='parsing error')}")
    out_path = create_next_model_folder('../llmOutputs')
    print(out_path) # ../llmOutputs/model1
    outs_df.to_csv(out_path+'/generations.csv', header=True, index=False)
else:
    if not PRIVATE:
        answers = data.answer.tolist()
        correct = sum([y==yhat for y,yhat in zip(answers,outs)])
        print(f'{correct} correct answers')    
    data['answer'] = outs
    data[['id','answer']].to_csv("submission.csv", header=True, index=False)

1 correct answers
